In [1]:
import numpy as np
import cv2
import time
from packages.number_detection.src.FC import NP_model

In [2]:
model_path = 'packages/number_detection/config/model_weights.npy'

weight_dict = np.load(model_path, allow_pickle=True).item()

model = NP_model(weight_dict)

In [3]:
HSV_MASK_LOW = (70,10,180)
HSV_MASK_HIGH = (90,200,255)

In [4]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [24]:

def process_image(image):
    start_time = time.time()

    image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    image_mask = cv2.inRange(image_hsv, HSV_MASK_LOW, HSV_MASK_HIGH)
    # opening = cv2.morphologyEx(image_mask, cv2.MORPH_OPEN, np.ones((5,5),np.uint8), iterations = 2)

    contours, _ = cv2.findContours(image_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) == 0:
        return image
    

    c = max(contours, key=cv2.contourArea)

    if cv2.contourArea(c) < 10000:
        return image

    number_bb = cv2.boundingRect(c)
    x = int(number_bb[0])
    y = int(number_bb[1])
    w = int(number_bb[2])
    h = int(number_bb[3])

    ar = float(w) / float(h)
    if ar < 0.66 or ar > 1.33:
        return image

    padding = int(min(h * 0.10, w * 0.10))

    number = image[y + padding:y+h - padding, x + padding:x + w - padding]


    black_max = (90,125,125)
    black_min = (0,0,0)
    number_mask = cv2.inRange(number, black_min, black_max)
    number_mask = cv2.resize(number_mask, (28,28))
    cv2.imshow("number_mask", number_mask)

    input_vector = number_mask.reshape(1, 28 * 28)
    res_vector = model.predict(input_vector)
    number = np.argmax(res_vector)

    end_time = time.time()

    duration = (end_time - start_time) * 1000
    duration_str = "{:.3f}".format(duration) + " ms"
    

    cv2.rectangle(image,(x, y), (x+w, y+h), (0,255,0), 2)
    cv2.putText(image, str(number), (x,y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,255,12), 2)
    cv2.putText(image, str(duration_str), (x + 30 ,y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255,255), 2)

    return image

    

In [25]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()

    processed_image = process_image(frame)

    cv2.imshow('Processed image', processed_image)
     
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()